# Direct Transcrition Methods
(Working in Progress) This tutorial follows optimization, and use direct transcription method to solve trajectory optimization problem.

The Nonlinear Programming Problem(NLP) is solve with help of casadi and IPOPT. 

In [1]:
import mujoco
import mediapy as media
from casadi import *

## Cartpole dynamics

In [2]:
xml = """"
<mujoco model="inverted pendulum">
  <compiler inertiafromgeom="true"/>
  <default>
    <joint armature="0" damping="1" limited="true"/>
    <geom contype="0" friction="1 0.1 0.1" rgba="0.4 0.33 0.26 1.0"/>
    <tendon/>
    <motor ctrlrange="-3 3"/>
  </default>
  <option gravity="0 0 -9.81" timestep="0.02" />
  <custom>
    <!-- brax custom params -->
    <numeric data="10000" name="constraint_stiffness"/>
    <numeric data="10000" name="constraint_limit_stiffness"/>
    <numeric data="0" name="spring_mass_scale"/>
    <numeric data="1" name="spring_inertia_scale"/>
    <numeric data="5" name="solver_maxls"/>
  </custom>
  <size nstack="3000"/>
  <worldbody>
    <light diffuse="1 1 1" pos="0 0 3" dir="0 0 -1"/>
    <!-- geom name="ground" type="plane" pos="0 0 0" / -->
    <geom name="rail" pos="0 0 0" quat="0.707 0 0.707 0" size="0.02 1" type="capsule" rgba="1 1 1 1"/>
    <body name="cart" pos="0 0 0">
      <joint axis="1 0 0" limited="false" name="slider" pos="0 0 0" type="slide"/>
      <geom name="cart" pos="0 0 0" quat="0.707 0 0.707 0" size="0.1 0.1" type="capsule"/>
      <body name="pole" pos="0 0 0">
        <joint axis="0 1 0" name="hinge" pos="0 0 0" limited="false" type="hinge"/>
        <geom fromto="0 0 0 0.001 0 0.6" name="cpole" size="0.049 0.3" type="capsule"/>
        <!--                 <body name="pole2" pos="0.001 0 0.6"><joint name="hinge2" type="hinge" pos="0 0 0" axis="0 1 0"/><geom name="cpole2" type="capsule" fromto="0 0 0 0 0 0.6" size="0.05 0.3" rgba="0.7 0 0.7 1"/><site name="tip2" pos="0 0 .6"/></body>-->
      </body>
    </body>
  </worldbody>
  <actuator>
    <motor ctrllimited="true" ctrlrange="-3 3" gear="100" joint="slider" name="slide"/>
  </actuator>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)
renderer = mujoco.Renderer(model)

duration = 3.8  # (seconds)
framerate = 60  # (Hz)

# Simulate and display video.
frames = []
mujoco.mj_resetData(model, data)  # Reset state and time.
while data.time < duration:
    mujoco.mj_step(model, data)
    if len(frames) < data.time * framerate:
        renderer.update_scene(data)
        pixels = renderer.render()
        frames.append(pixels)
media.show_video(frames, fps=framerate)

This browser does not support the video tag.


## Casadi Wrap for mujoco dynamics

In [3]:
class CasadiMujocoWrap(Callback):
    def __init__(self, name, opts={}):
        Callback.__init__(self)
        self.construct(name, opts)

    # Number of inputs and outputs
    def get_n_in(self): return 2
    def get_n_out(self): return 1

    def get_sparsity_in(self,i):
        if i == 0:
            return Sparsity.dense(4,1)
        elif i == 1:
            return Sparsity.dense(1,1)
        else:
            raise ValueError("wrong args index")

    def get_sparsity_out(self,i):
        return Sparsity.dense(4,1)

    # Evaluate numerically
    def eval(self, arg):
        x = arg[0].full().flatten()
        u = arg[1].full().flatten()
        qpos, qvel = np.split(x, 2)
        data.qpos = qpos
        data.qvel = qvel
        data.ctrl = u
        mujoco.mj_step(model, data)
        x_next = np.hstack([data.qpos, data.qvel])
        return [DM(x_next)]

mujoco_dyanmics = CasadiMujocoWrap('f', {"enable_fd":True})

In [4]:
mujoco.mj_resetData(model, data) 

state = np.hstack([data.qpos, data.qvel])
u = np.array([1.0])
frames = []
while data.time < duration:
    state = mujoco_dyanmics(state, u).full().flatten()
    data.qpos = state[:2]
    data.qvel = state[2:]
    mujoco.mj_forward(model, data)
    if len(frames) < data.time * framerate:
        renderer.update_scene(data)
        pixels = renderer.render()
        frames.append(pixels)
media.show_video(frames, fps=framerate)

## Construct NLP and solve with IPOPT

In [5]:
T = 50

prob = Opti()
x = prob.variable(4, T+1)
u = prob.variable(1, T)

# initial condition
x0 = np.array([0, np.pi, 0, 0])
# target
x_target = np.array([0, 0, 0, 0])
# initial condition constraints
prob.subject_to(x[:, 0] == x0)
# terminal condition
prob.subject_to(x[:, -1] == x_target)
for i in range(T):
    # dynamics constraints
    prob.subject_to(x[:, i+1] == mujoco_dyanmics(x[:, i], u[:, i]))
# control limit
prob.subject_to([u>=-3, u<=3])
# just feasible problem, no objective
prob.minimize(0)

# set initial guess (it is crtical to obtain the solution in current NLP formulation)
x_init = np.swapaxes(np.linspace(x0, x_target, T+1), 0, 1)
prob.set_initial(x, x_init)

prob.solver("ipopt")
sol = prob.solve()
traj_x = np.swapaxes(sol.value(x), 0, 1)
traj_u = sol.value(u)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:     1208
Number of nonzeros in inequality constraint Jacobian.:      100
Number of nonzeros in Lagrangian Hessian.............:      750

Total number of variables............................:      254
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      208
Total number of inequality c

## Test results

In [6]:
frames = []
for state in traj_x:
    data.qpos = state[:2]
    data.qvel = state[2:]
    mujoco.mj_forward(model, data)
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)
media.show_video(frames, fps=1/0.02)

In [7]:
mujoco.mj_resetData(model, data) 
state = x0
frames = []
for u in traj_u:
    state = mujoco_dyanmics(state, u).full().flatten()
    data.qpos = state[:2]
    data.qvel = state[2:]
    mujoco.mj_forward(model, data)
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)
media.show_video(frames, fps=1/0.02)

## More NLPs

In [8]:
T = 50

prob = Opti()
x = prob.variable(4, T+1)
u = prob.variable(1, T)

# initial condition
x0 = np.array([0, np.pi, 0, 0])
# initial condition constraints
prob.subject_to(x[:, 0] == x0)

# target
x_target = np.array([0, 0, 0, 0])
# we do not set terminal condition but using running cost to push carpole up
Q = diag(SX([0.01, 1, 0.01, 0.01]))
R = diag(SX([0.01]))
x_sym = SX.sym("x", 4)
u_sym = SX.sym("u", 1)
cost_fn = Function('cost', [x_sym, u_sym], [(x_sym-x_target).T @ Q @ (x_sym-x_target) + u_sym.T @ R @ u_sym])
J = 0
for i in range(T):
    # dynamics constraints
    prob.subject_to(x[:, i+1] == mujoco_dyanmics(x[:, i], u[:, i]))
    J = J + cost_fn(x[:, i], u[:, i])
# control limit
prob.subject_to([u>=-3, u<=3])
# minimize running cost
prob.minimize(J)

# set initial guess (it is crtical to obtain the solution in current NLP formulation)
x_init = np.swapaxes(np.linspace(x0, x_target, T+1), 0, 1)
prob.set_initial(x, x_init)
prob.solver("ipopt", {}, {"max_iter": 100})

try:
    sol = prob.solve()
    traj_x = np.swapaxes(sol.value(x), 0, 1)
    traj_u = sol.value(u)
except:
    traj_x = np.swapaxes(prob.debug.value(x), 0, 1)
    traj_u = prob.debug.value(u)
    print("Solver fails return last iteration results!")


This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:     1204
Number of nonzeros in inequality constraint Jacobian.:      100
Number of nonzeros in Lagrangian Hessian.............:      750

Total number of variables............................:      254
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      204
Total number of inequality constraints...............:      100
        inequality constraints with only lower bounds:       50
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       50

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.6946111e+02 4.47e-01 7.47e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [9]:
frames = []
for state in traj_x:
    data.qpos = state[:2]
    data.qvel = state[2:]
    mujoco.mj_forward(model, data)
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)
media.show_video(frames, fps=1/0.02)

In [10]:
mujoco.mj_resetData(model, data) 
state = x0
frames = []
for u in traj_u:
    state = mujoco_dyanmics(state, u).full().flatten()
    data.qpos = state[:2]
    data.qvel = state[2:]
    mujoco.mj_forward(model, data)
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)
media.show_video(frames, fps=1/0.02)

## Try SNOPT solver

This require SNOPT to be installed with casadi.

In [11]:
T = 50

prob = Opti()
x = prob.variable(4, T+1)
u = prob.variable(1, T)

# initial condition
x0 = np.array([0, np.pi, 0, 0])
# target
x_target = np.array([0, 0, 0, 0])
# initial condition constraints
prob.subject_to(x[:, 0] == x0)
# terminal condition
prob.subject_to(x[:, -1] == x_target)
for i in range(T):
    # dynamics constraints
    prob.subject_to(x[:, i+1] == mujoco_dyanmics(x[:, i], u[:, i]))
# control limit
prob.subject_to([u>=-3, u<=3])
# just feasible problem, no objective
prob.minimize(0)

# set initial guess (it is crtical to obtain the solution in current NLP formulation)
x_init = np.swapaxes(np.linspace(x0, x_target, T+1), 0, 1)
prob.set_initial(x, x_init)
prob.solver("snopt")

try:
    sol = prob.solve()
    traj_x = np.swapaxes(sol.value(x), 0, 1)
    traj_u = sol.value(u)
except:
    traj_x = np.swapaxes(prob.debug.value(x), 0, 1)
    traj_u = prob.debug.value(u)
    print("Solver fails!")


    SNOPT  C interface  2.2.0   
 S N O P T  7.7.7    (Feb 2021)

 SNMEMB EXIT 100 -- finished successfully
 SNMEMB INFO 104 -- memory requirements estimated

 Trial version of SNOPT -- for evaluation or academic purposes only



 Nonlinear constraints     308     Linear constraints       0
 Nonlinear variables       254     Linear variables         0
 Jacobian  variables       254     Objective variables    254
 Total constraints         308     Total variables        254



 The user has defined    1308   out of    1308   constraint gradients.
 The user has defined     254   out of     254   objective  gradients.


 Major Minors     Step   nCon Feasible  Optimal  MeritFunction     nS Penalty
     0     87               1  1.4E-01  3.3E-01  0.0000000E+00     10           r i
     1     34  1.3E-01      2  1.2E-01  3.1E-01  3.9130122E+05     24 8.4E+05   rli
     2     24  1.9E-01      4  9.6E-02  2.6E-01  3.2248337E+05     15 8.4E+05   rli
     3     18  8.2E-01      7  8.0E-02  3.9E-

In [12]:
frames = []
for state in traj_x:
    data.qpos = state[:2]
    data.qvel = state[2:]
    mujoco.mj_forward(model, data)
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)
media.show_video(frames, fps=1/0.02)

In [13]:
mujoco.mj_resetData(model, data) 
state = x0
frames = []
for u in traj_u:
    state = mujoco_dyanmics(state, u).full().flatten()
    data.qpos = state[:2]
    data.qvel = state[2:]
    mujoco.mj_forward(model, data)
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)
media.show_video(frames, fps=1/0.02)

In [14]:
T = 50

prob = Opti()
x = prob.variable(4, T+1)
u = prob.variable(1, T)

# initial condition
x0 = np.array([0, np.pi, 0, 0])
# initial condition constraints
prob.subject_to(x[:, 0] == x0)

# target
x_target = np.array([0, 0, 0, 0])
# we do not set terminal condition but using running cost to push carpole up
Q = diag(SX([0.01, 1, 0.01, 0.01]))
R = diag(SX([0.01]))
x_sym = SX.sym("x", 4)
u_sym = SX.sym("u", 1)
cost_fn = Function('cost', [x_sym, u_sym], [(x_sym-x_target).T @ Q @ (x_sym-x_target) + u_sym.T @ R @ u_sym])
J = 0
for i in range(T):
    # dynamics constraints
    prob.subject_to(x[:, i+1] == mujoco_dyanmics(x[:, i], u[:, i]))
    J = J + cost_fn(x[:, i], u[:, i])
# control limit
prob.subject_to([u>=-3, u<=3])
# minimize running cost
prob.minimize(J)

# set initial guess (it is crtical to obtain the solution in current NLP formulation)
x_init = np.swapaxes(np.linspace(x0, x_target, T+1), 0, 1)
prob.set_initial(x, x_init)
prob.solver("snopt", {}, {"max_iter": 100})

try:
    sol = prob.solve()
    traj_x = np.swapaxes(sol.value(x), 0, 1)
    traj_u = sol.value(u)
except:
    traj_x = np.swapaxes(prob.debug.value(x), 0, 1)
    traj_u = prob.debug.value(u)
    print("Solver fails return last iteration results!")


    SNOPT  C interface  2.2.0   
 S N O P T  7.7.7    (Feb 2021)

 SNMEMB EXIT 100 -- finished successfully
 SNMEMB INFO 104 -- memory requirements estimated

 Trial version of SNOPT -- for evaluation or academic purposes only



 Nonlinear constraints     304     Linear constraints       0
 Nonlinear variables       254     Linear variables         0
 Jacobian  variables       254     Objective variables    254
 Total constraints         304     Total variables        254



 The user has defined    1304   out of    1304   constraint gradients.
 The user has defined     254   out of     254   objective  gradients.


 Major Minors     Step   nCon Feasible  Optimal  MeritFunction     nS Penalty
     0     77               1  1.4E-01  2.4E-02  1.6946111E+02     34           r
     1      8  2.0E-01      2  1.2E-01  2.1E-02 -7.1242571E+03     29 1.7E+04   rl
     2     11  2.2E-01      3  9.8E-02  3.8E-02 -5.4218832E+03     33 1.7E+04 s  l
     3     19  1.0E+00      5  7.1E-02  1.0E+00  

In [15]:
frames = []
for state in traj_x:
    data.qpos = state[:2]
    data.qvel = state[2:]
    mujoco.mj_forward(model, data)
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)
media.show_video(frames, fps=1/0.02)

In [16]:
mujoco.mj_resetData(model, data) 
state = x0
frames = []
for u in traj_u:
    state = mujoco_dyanmics(state, u).full().flatten()
    data.qpos = state[:2]
    data.qvel = state[2:]
    mujoco.mj_forward(model, data)
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)
media.show_video(frames, fps=1/0.02)

## Trajectory optimization embeded on MPC formulation


In [17]:
# target
x_target = np.array([0, 0, 0, 0])
# we do not set terminal condition but using running cost to push carpole up
Q = diag(SX([0.01, 1, 0.01, 0.01]))
R = diag(SX([0.01]))
x_sym = SX.sym("x", 4)
u_sym = SX.sym("u", 1)
cost_fn = Function('cost', [x_sym, u_sym], [(x_sym-x_target).T @ Q @ (x_sym-x_target) + u_sym.T @ R @ u_sym])

# Define construct sub-loop problem
def solve_mpc_sub_loop(x0, cost_fn, init_guess=None, T=50, solver="ipopt", max_iter=10, print_level=0):
    prob = Opti()
    x = prob.variable(4, T+1)
    u = prob.variable(1, T)

    # initial condition constraints
    prob.subject_to(x[:, 0] == x0)

    # control limit
    prob.subject_to([u>=-3, u<=3])

    J = 0
    
    for i in range(T):
        # dynamics constraints
        prob.subject_to(x[:, i+1] == mujoco_dyanmics(x[:, i], u[:, i]))
        J = J + cost_fn(x[:, i], u[:, i])
    # minimize running cost
    prob.minimize(J)

    if init_guess is not None:
        # NOTE currently lam is not updated
        if "x" in init_guess:
            print("Setting the init for x")
            prob.set_initial(x, init_guess["x"])
        if "u" in init_guess:
            print("Settiing the init for u")
            prob.set_initial(u, init_guess["u"])

    prob.solver(solver, {}, {"max_iter": max_iter, "print_level": print_level})
    try:
        sol = prob.solve()
        traj_x = sol.value(x)
        traj_u = sol.value(u)
        if solver=="ipopt":
            print("Solve with iter {:}".format(prob.stats()["iter_count"]))
        else:
            print("Solve successfully")
    except:
        traj_x = prob.debug.value(x)
        traj_u = prob.debug.value(u)
        if solver=="ipopt":
            print("Solver fails under iter {:} return last iteration results!".format(prob.stats()["iter_count"]))
        else:
            print("Solver fail")

    return {"x": traj_x, "u": traj_u}

# Update the initial guess for new MPC
def update_init_guess(last_results):
    x = last_results["x"]
    u = last_results["u"]

    x_last = x[:, -1]
    u_last = u[-1]

    # Keep the new u_last same as previous u_last
    u_new_last = np.copy(u_last)
    # x initial guess by step dynamics for one step
    x_new_last = mujoco_dyanmics(x_last, u_new_last)

    x_new_guess = np.concatenate([x[:, 1:], x_new_last], axis=1)
    u_new_guess = np.append(u[1:], u_new_last)

    return {"x": x_new_guess, "u":u_new_guess}

### MPC with IPOPT

In [ ]:
T = 50

x0 = np.array([0, np.pi, 0, 0])
initial_guess = {"x": np.swapaxes(np.linspace(x0, x_target, T+1), 0, 1)}

# obtain the first result
res = solve_mpc_sub_loop(x0=x0, cost_fn=cost_fn, init_guess=initial_guess, T=50)

frames = []
# mpc loop
for i in range(100):
    # execute the first control at solution
    x0 = mujoco_dyanmics(x0, res["u"][0]).full().flatten()

    # update initial guess
    initial_guess = update_init_guess(res)
    # solving sub-prob for new initial condition
    res = solve_mpc_sub_loop(x0=x0, cost_fn=cost_fn, init_guess=initial_guess, T=50, max_iter=2)

    # visualization
    data.qpos = x0[:2]
    data.qvel = x0[2:]
    mujoco.mj_forward(model, data)
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)

media.show_video(frames, fps=1/0.02)

Setting the init for x
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 179.00us ( 16.27us) 172.63us ( 15.69us)        11
       nlp_g  |  22.37ms (  2.03ms)  22.44ms (  2.04ms)        11
  nlp_grad_f  | 281.00us ( 23.42us) 282.45us ( 23.54us)        12
  nlp_hess_l  |   8.19 s (818.69ms)   8.20 s (819.56ms)        10
   nlp_jac_g  | 473.18ms ( 39.43ms) 473.40ms ( 39.45ms)        12
       total  |   8.70 s (  8.70 s)   8.71 s (  8.71 s)         1
Solver fails under iter 10 return last iteration results!
Setting the init for x
Settiing the init for u
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  66.00us ( 16.50us)  61.02us ( 15.26us)         4
       nlp_g  |   7.80ms (  1.95ms)   7.81ms (  1.95ms)         4
  nlp_grad_f  |  98.00us ( 24.50us)  97.70us ( 24.43us)         4
  nlp_hess_l  |   1.59 s (795.84ms)   1.59 s (796.01ms)         2
   nlp_jac_g  | 153.51ms ( 38.38ms) 153.60ms ( 38.40ms)         4
       total  

### MPC with SNOPT

In [ ]:
T = 50

x0 = np.array([0, np.pi, 0, 0])
initial_guess = {"x": np.swapaxes(np.linspace(x0, x_target, T+1), 0, 1)}

# obtain the first result
res = solve_mpc_sub_loop(x0=x0, cost_fn=cost_fn, init_guess=initial_guess, T=50, solver="snopt")

frames = []
# mpc loop
for i in range(100):
    # execute the first control at solution
    x0 = mujoco_dyanmics(x0, res["u"][0]).full().flatten()

    # update initial guess
    initial_guess = update_init_guess(res)
    # solving sub-prob for new initial condition
    res = solve_mpc_sub_loop(x0=x0, cost_fn=cost_fn, init_guess=initial_guess, T=50, solver="snopt", max_iter=2)

    # visualization
    data.qpos = x0[:2]
    data.qvel = x0[2:]
    mujoco.mj_forward(model, data)
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)

media.show_video(frames, fps=1/0.02)

Setting the init for x
    SNOPT  C interface  2.2.0   
 S N O P T  7.7.7    (Feb 2021)

 SNMEMB EXIT 100 -- finished successfully
 SNMEMB INFO 104 -- memory requirements estimated

 Trial version of SNOPT -- for evaluation or academic purposes only



 Nonlinear constraints     304     Linear constraints       0
 Nonlinear variables       254     Linear variables         0
 Jacobian  variables       254     Objective variables    254
 Total constraints         304     Total variables        254



 The user has defined    1304   out of    1304   constraint gradients.
 The user has defined     254   out of     254   objective  gradients.


 Major Minors     Step   nCon Feasible  Optimal  MeritFunction     nS Penalty
     0     77               1  1.4E-01  2.4E-02  1.6946111E+02     34           r
     1      8  2.0E-01      2  1.2E-01  2.1E-02 -7.1242571E+03     29 1.7E+04   rl
     2     11  2.2E-01      3  9.8E-02  3.8E-02 -5.4218832E+03     33 1.7E+04 s  l
     3     19  1.0E+00    

This browser does not support the video tag.


### MPC IPOPT with zero initial guess for first iter

In [ ]:
T = 50

x0 = np.array([0, np.pi, 0, 0])

# obtain the first result
res = solve_mpc_sub_loop(x0=x0, cost_fn=cost_fn, T=50)

frames = []
# mpc loop
for i in range(100):
    # execute the first control at solution
    x0 = mujoco_dyanmics(x0, res["u"][0]).full().flatten()

    # update initial guess
    initial_guess = update_init_guess(res)
    # solving sub-prob for new initial condition
    res = solve_mpc_sub_loop(x0=x0, cost_fn=cost_fn, init_guess=initial_guess, T=50, max_iter=2)

    # visualization
    data.qpos = x0[:2]
    data.qvel = x0[2:]
    mujoco.mj_forward(model, data)
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)

media.show_video(frames, fps=1/0.02)

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 694.00us ( 11.02us) 691.55us ( 10.98us)        63
       nlp_g  | 109.48ms (  1.74ms) 109.30ms (  1.73ms)        63
  nlp_grad_f  | 272.00us ( 22.67us) 272.04us ( 22.67us)        12
  nlp_hess_l  |   7.48 s (748.47ms)   7.49 s (749.10ms)        10
   nlp_jac_g  | 425.79ms ( 35.48ms) 419.73ms ( 34.98ms)        12
       total  |   8.04 s (  8.04 s)   8.04 s (  8.04 s)         1
{'iter_count': 10, 'iterations': {'alpha_du': [0.0, 0.1963517338669116, 0.3293432841705036, 0.31653691580583876, 0.2863947222746829, 0.24429999229372937, 0.19730048454099178, 0.16569926787165273, 0.13956759145089342, 0.11988821150823215, 0.10188405492206452], 'alpha_pr': [0.0, 0.12146328314123404, 0.04311648879999081, 0.03183474883023792, 0.03198674167341317, 0.027164822568032647, 0.01072322226814846, 0.009236040441646911, 0.007806486617809978, 0.00666771971908206, 0.005652154006847556], 'd_norm': [0.0, 12.225916972154328, 12.070117